# Modelo de Suavización Exponencial - Promedio Móvil Ponderado (WMA)

Este notebook implementa el método de Promedio Móvil Ponderado (WMA - Weighted Moving Average) para predecir las ventas de dos productos, utilizando tres tipos de validación temporal:
- Walk-Forward Validation
- Rolling Window
- Expanding Window


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función WMA (Weighted Moving Average)  

Implementación del método de Promedio Móvil Ponderado para realizar predicciones. En WMA, los valores más recientes tienen mayor peso.


In [3]:
def wma_forecast(data, window_size, forecast_horizon=1):
    """
    Calcula el promedio móvil ponderado y realiza predicciones.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    window_size : int
        Tamaño de la ventana para el promedio móvil ponderado
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    
    Returns:
    --------
    float
        Predicción usando WMA
    """
    if len(data) < window_size:
        # Si no hay suficientes datos, usar el promedio de todos los disponibles
        return np.mean(data)
    
    # Obtener los últimos window_size valores
    window_data = data[-window_size:]
    
    # Crear pesos: valores más recientes tienen mayor peso
    # Pesos lineales: 1, 2, 3, ..., window_size
    weights = np.arange(1, window_size + 1)
    
    # Normalizar los pesos para que sumen 1
    weights = weights / weights.sum()
    
    # Calcular el promedio móvil ponderado
    wma_value = np.sum(window_data * weights)
    
    # Para WMA, todas las predicciones futuras son iguales al último WMA calculado
    return wma_value


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [4]:
def walk_forward_validation(dataset, window_size=5, train_size_min=10):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    window_size : int
        Tamaño de la ventana para WMA (por defecto 5)
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            prediction = wma_forecast(train_data, window_size, forecast_horizon=1)
            
            # Calcular métricas
            error = actual - prediction
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana_train': i,
                'indice_test': i,
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 2: Rolling Window Validation

En Rolling Window Validation, se mantiene un tamaño fijo de ventana de entrenamiento que se desplaza a lo largo del tiempo.


In [5]:
def rolling_window_validation(dataset, window_size=5, train_window_size=20):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    window_size : int
        Tamaño de la ventana para WMA (por defecto 5)
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            prediction = wma_forecast(train_data, window_size, forecast_horizon=1)
            
            # Calcular métricas
            error = actual - prediction
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana_train_inicio': i - train_window_size,
                'ventana_train_fin': i,
                'indice_test': i,
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [6]:
def expanding_window_validation(dataset, window_size=5, train_size_min=10):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    window_size : int
        Tamaño de la ventana para WMA (por defecto 5)
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            prediction = wma_forecast(train_data, window_size, forecast_horizon=1)
            
            # Calcular métricas
            error = actual - prediction
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana_train': i,  # Tamaño de la ventana que se expande
                'indice_test': i,
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo teniendo en cuenta los tres tipos de validación, ahora ejecutamos las tres validaciones para cada producto y calculamos las métricas promedio.


In [7]:
# Parámetros de validación
WINDOW_SIZE = 5  # Tamaño de ventana para WMA
TRAIN_SIZE_MIN = 10  # Tamaño mínimo de entrenamiento
TRAIN_WINDOW_SIZE = 20  # Tamaño fijo para Rolling Window

# Ejecutar Walk-Forward Validation
print("=" * 60)
print("WALK-FORWARD VALIDATION")
print("=" * 60)
results_wf = walk_forward_validation(df, window_size=WINDOW_SIZE, train_size_min=TRAIN_SIZE_MIN)


WALK-FORWARD VALIDATION

=== Walk-Forward Validation para producto1 ===
  Iteración 20/117: RMSE=7.6285, MAE=7.6285
  Iteración 40/117: RMSE=14.8007, MAE=14.8007
  Iteración 60/117: RMSE=7.8917, MAE=7.8917
  Iteración 80/117: RMSE=15.2534, MAE=15.2534
  Iteración 100/117: RMSE=22.9305, MAE=22.9305
  Total de iteraciones: 117

=== Walk-Forward Validation para producto2 ===
  Iteración 20/117: RMSE=29.6763, MAE=29.6763
  Iteración 40/117: RMSE=30.7583, MAE=30.7583
  Iteración 60/117: RMSE=15.9860, MAE=15.9860
  Iteración 80/117: RMSE=9.2763, MAE=9.2763
  Iteración 100/117: RMSE=44.3935, MAE=44.3935
  Total de iteraciones: 117


In [8]:
# Ejecutar Rolling Window Validation
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)
results_rw = rolling_window_validation(df, window_size=WINDOW_SIZE, train_window_size=TRAIN_WINDOW_SIZE)



ROLLING WINDOW VALIDATION

=== Rolling Window Validation para producto1 ===
  Iteración 20/107: RMSE=16.6700, MAE=16.6700
  Iteración 40/107: RMSE=19.3908, MAE=19.3908
  Iteración 60/107: RMSE=2.6011, MAE=2.6011
  Iteración 80/107: RMSE=13.4138, MAE=13.4138
  Iteración 100/107: RMSE=10.4969, MAE=10.4969
  Total de iteraciones: 107

=== Rolling Window Validation para producto2 ===
  Iteración 20/107: RMSE=1.1148, MAE=1.1148
  Iteración 40/107: RMSE=27.4116, MAE=27.4116
  Iteración 60/107: RMSE=10.6324, MAE=10.6324
  Iteración 80/107: RMSE=5.4955, MAE=5.4955
  Iteración 100/107: RMSE=3.1738, MAE=3.1738
  Total de iteraciones: 107


In [9]:
# Ejecutar Expanding Window Validation
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)
results_ew = expanding_window_validation(df, window_size=WINDOW_SIZE, train_size_min=TRAIN_SIZE_MIN)



EXPANDING WINDOW VALIDATION

=== Expanding Window Validation para producto1 ===
  Iteración 20/117: RMSE=7.6285, MAE=7.6285
  Iteración 40/117: RMSE=14.8007, MAE=14.8007
  Iteración 60/117: RMSE=7.8917, MAE=7.8917
  Iteración 80/117: RMSE=15.2534, MAE=15.2534
  Iteración 100/117: RMSE=22.9305, MAE=22.9305
  Total de iteraciones: 117

=== Expanding Window Validation para producto2 ===
  Iteración 20/117: RMSE=29.6763, MAE=29.6763
  Iteración 40/117: RMSE=30.7583, MAE=30.7583
  Iteración 60/117: RMSE=15.9860, MAE=15.9860
  Iteración 80/117: RMSE=9.2763, MAE=9.2763
  Iteración 100/117: RMSE=44.3935, MAE=44.3935
  Total de iteraciones: 117


In [10]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        # Calcular RMSE global (sobre todas las predicciones)
        actuales = [m['actual'] for m in metrics_list]
        predicciones = [m['prediccion'] for m in metrics_list]
        rmse_global = np.sqrt(mean_squared_error(actuales, predicciones))
        mae_global = mean_absolute_error(actuales, predicciones)
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'rmse_global': rmse_global,
            'mae_global': mae_global,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio (por iteración): {rmse_promedio:.4f}")
        print(f"  MAE Promedio (por iteración): {mae_promedio:.4f}")
        print(f"  RMSE Global (sobre todas las predicciones): {rmse_global:.4f}")
        print(f"  MAE Global (sobre todas las predicciones): {mae_global:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
metricas_wf = calcular_metricas_promedio(results_wf, "Walk-Forward")
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - WALK-FORWARD

producto1:
  RMSE Promedio (por iteración): 13.5918
  MAE Promedio (por iteración): 13.5918
  RMSE Global (sobre todas las predicciones): 16.3214
  MAE Global (sobre todas las predicciones): 13.5918
  Número de iteraciones: 117

producto2:
  RMSE Promedio (por iteración): 19.0951
  MAE Promedio (por iteración): 19.0951
  RMSE Global (sobre todas las predicciones): 22.4406
  MAE Global (sobre todas las predicciones): 19.0951
  Número de iteraciones: 117

MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio (por iteración): 14.2463
  MAE Promedio (por iteración): 14.2463
  RMSE Global (sobre todas las predicciones): 16.8727
  MAE Global (sobre todas las predicciones): 14.2463
  Número de iteraciones: 107

producto2:
  RMSE Promedio (por iteración): 19.0766
  MAE Promedio (por iteración): 19.0766
  RMSE Global (sobre todas las predicciones): 22.5522
  MAE Global (sobre todas las predicciones): 19.0766
  Número de iteraciones: 107

MÉTRICAS PROM

## Comparación de Resultados y Selección del Mejor Modelo

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [11]:
# Crear DataFrame comparativo
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Walk-Forward',
        'RMSE_Promedio': metricas_wf[producto]['rmse_promedio'],
        'RMSE_Global': metricas_wf[producto]['rmse_global'],
        'MAE_Promedio': metricas_wf[producto]['mae_promedio'],
        'MAE_Global': metricas_wf[producto]['mae_global']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'RMSE_Global': metricas_rw[producto]['rmse_global'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio'],
        'MAE_Global': metricas_rw[producto]['mae_global']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'RMSE_Global': metricas_ew[producto]['rmse_global'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio'],
        'MAE_Global': metricas_ew[producto]['mae_global']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "="*80)
print("COMPARACIÓN DE RESULTADOS")
print("="*80)
print(df_comparacion.to_string(index=False))



COMPARACIÓN DE RESULTADOS
 Producto       Validacion  RMSE_Promedio  RMSE_Global  MAE_Promedio  MAE_Global
producto1     Walk-Forward      13.591779    16.321390     13.591779   13.591779
producto1   Rolling Window      14.246256    16.872683     14.246256   14.246256
producto1 Expanding Window      13.591779    16.321390     13.591779   13.591779
producto2     Walk-Forward      19.095107    22.440563     19.095107   19.095107
producto2   Rolling Window      19.076564    22.552173     19.076564   19.076564
producto2 Expanding Window      19.095107    22.440563     19.095107   19.095107


In [12]:
# Encontrar el mejor RMSE para cada producto
print("\n" + "="*80)
print("MEJOR MÉTODO POR PRODUCTO (basado en RMSE Global)")
print("="*80)

mejores_metodos = {}

for producto in df.columns:
    # Comparar RMSE global de cada método
    rmse_wf = metricas_wf[producto]['rmse_global']
    rmse_rw = metricas_rw[producto]['rmse_global']
    rmse_ew = metricas_ew[producto]['rmse_global']
    
    metodos = {
        'Walk-Forward': rmse_wf,
        'Rolling Window': rmse_rw,
        'Expanding Window': rmse_ew
    }
    
    mejor_metodo = min(metodos, key=metodos.get)
    mejor_rmse = metodos[mejor_metodo]
    
    mejores_metodos[producto] = {
        'metodo': mejor_metodo,
        'rmse': mejor_rmse,
        'mae': metricas_wf[producto]['mae_global'] if mejor_metodo == 'Walk-Forward' else \
               metricas_rw[producto]['mae_global'] if mejor_metodo == 'Rolling Window' else \
               metricas_ew[producto]['mae_global']
    }
    
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor_metodo}")
    print(f"  RMSE Global: {mejor_rmse:.4f}")
    print(f"  MAE Global: {mejores_metodos[producto]['mae']:.4f}")
    print(f"\n  Comparación:")
    print(f"    Walk-Forward:     RMSE = {rmse_wf:.4f}")
    print(f"    Rolling Window:  RMSE = {rmse_rw:.4f}")
    print(f"    Expanding Window: RMSE = {rmse_ew:.4f}")

print("\n" + "="*80)



MEJOR MÉTODO POR PRODUCTO (basado en RMSE Global)

producto1:
  Mejor método: Walk-Forward
  RMSE Global: 16.3214
  MAE Global: 13.5918

  Comparación:
    Walk-Forward:     RMSE = 16.3214
    Rolling Window:  RMSE = 16.8727
    Expanding Window: RMSE = 16.3214

producto2:
  Mejor método: Walk-Forward
  RMSE Global: 22.4406
  MAE Global: 19.0951

  Comparación:
    Walk-Forward:     RMSE = 22.4406
    Rolling Window:  RMSE = 22.5522
    Expanding Window: RMSE = 22.4406



## Modelo Final para Producción

Entrenamos el modelo final usando todos los datos disponibles y el mejor método de validación para realizar predicciones de producción.


In [13]:
def modelo_final_produccion(dataset, window_size=5, horizonte_prediccion=1):
    """
    Entrena el modelo final usando todos los datos disponibles y realiza predicciones.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset completo con todos los datos históricos
    window_size : int
        Tamaño de la ventana para WMA
    horizonte_prediccion : int
        Número de períodos futuros a predecir
    
    Returns:
    --------
    dict
        Diccionario con predicciones para cada producto
    """
    predicciones = {}
    
    print("="*80)
    print("MODELO FINAL PARA PRODUCCIÓN")
    print("="*80)
    
    for producto in dataset.columns:
        print(f"\n{producto}:")
        data = dataset[producto].values
        
        # Usar todos los datos disponibles para entrenar
        # Para WMA, calculamos el promedio ponderado de los últimos window_size valores
        window_data = data[-window_size:]
        weights = np.arange(1, window_size + 1)
        weights = weights / weights.sum()
        wma_final = np.sum(window_data * weights)
        
        # Realizar predicciones futuras
        # Para WMA, todas las predicciones futuras son iguales al último WMA calculado
        predicciones_futuras = [wma_final] * horizonte_prediccion
        
        predicciones[producto] = {
            'wma_value': wma_final,
            'predicciones': predicciones_futuras,
            'window_size': window_size,
            'ultimos_valores': data[-window_size:].tolist(),
            'pesos': weights.tolist(),
            'ultimo_valor_real': data[-1]
        }
        
        print(f"  Tamaño de ventana WMA: {window_size}")
        print(f"  Últimos {window_size} valores: {data[-window_size:]}")
        print(f"  Pesos aplicados: {weights}")
        print(f"  WMA calculado: {wma_final:.4f}")
        print(f"  Último valor real: {data[-1]:.4f}")
        print(f"  Predicción para el siguiente período: {wma_final:.4f}")
        if horizonte_prediccion > 1:
            print(f"  Predicciones para los próximos {horizonte_prediccion} períodos: {predicciones_futuras}")
    
    return predicciones

# Entrenar modelo final y realizar predicciones
predicciones_finales = modelo_final_produccion(df, window_size=WINDOW_SIZE, horizonte_prediccion=1)


MODELO FINAL PARA PRODUCCIÓN

producto1:
  Tamaño de ventana WMA: 5
  Últimos 5 valores: [164.6107711 150.8818393 151.7884696 137.0476394 141.9908727]
  Pesos aplicados: [0.06666667 0.13333333 0.2        0.26666667 0.33333333]
  WMA calculado: 145.3257
  Último valor real: 141.9909
  Predicción para el siguiente período: 145.3257

producto2:
  Tamaño de ventana WMA: 5
  Últimos 5 valores: [629.2930339 637.0994671 653.1552818 672.5283447 676.0580921]
  Pesos aplicados: [0.06666667 0.13333333 0.2        0.26666667 0.33333333]
  WMA calculado: 662.2241
  Último valor real: 676.0581
  Predicción para el siguiente período: 662.2241


## Resumen Final

Resumen de los resultados obtenidos con el método WMA.


In [14]:
print("\n" + "="*80)
print("RESUMEN FINAL")
print("="*80)

print("\nParámetros utilizados:")
print(f"  Tamaño de ventana WMA: {WINDOW_SIZE}")
print(f"  Tamaño mínimo de entrenamiento: {TRAIN_SIZE_MIN}")
print(f"  Tamaño de ventana fija (Rolling): {TRAIN_WINDOW_SIZE}")

print("\nMejor método de validación por producto:")
for producto, info in mejores_metodos.items():
    print(f"\n  {producto}:")
    print(f"    Método: {info['metodo']}")
    print(f"    RMSE Global: {info['rmse']:.4f}")
    print(f"    MAE Global: {info['mae']:.4f}")

print("\nPredicciones finales para producción:")
for producto, pred in predicciones_finales.items():
    print(f"\n  {producto}:")
    print(f"    Predicción siguiente período: {pred['predicciones'][0]:.4f}")
    print(f"    Basado en WMA de los últimos {pred['window_size']} valores")



RESUMEN FINAL

Parámetros utilizados:
  Tamaño de ventana WMA: 5
  Tamaño mínimo de entrenamiento: 10
  Tamaño de ventana fija (Rolling): 20

Mejor método de validación por producto:

  producto1:
    Método: Walk-Forward
    RMSE Global: 16.3214
    MAE Global: 13.5918

  producto2:
    Método: Walk-Forward
    RMSE Global: 22.4406
    MAE Global: 19.0951

Predicciones finales para producción:

  producto1:
    Predicción siguiente período: 145.3257
    Basado en WMA de los últimos 5 valores

  producto2:
    Predicción siguiente período: 662.2241
    Basado en WMA de los últimos 5 valores


## Exportación de Métricas

Exportamos las métricas de cada validación para análisis posterior.


In [15]:
# Exportar métricas detalladas a DataFrames
def exportar_metricas(results, nombre_validacion):
    """
    Exporta las métricas a un DataFrame.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con DataFrames por producto
    """
    dataframes = {}
    
    for producto in results.keys():
        metrics_list = results[producto]
        df_metrics = pd.DataFrame(metrics_list)
        dataframes[producto] = df_metrics
        
        # Guardar en CSV
        filename = f'metricas_{nombre_validacion.replace(" ", "_")}_{producto}.csv'
        df_metrics.to_csv("metricas/"+"modelo-WMA-"+filename, index=False)
        print(f"Métricas de {producto} ({nombre_validacion}) exportadas a: metricas/modelo-WMA-{filename}")
    
    return dataframes

# Crear directorio de métricas si no existe
import os
os.makedirs('metricas', exist_ok=True)

# Exportar todas las métricas
print("\nExportando métricas...")
metricas_wf_df = exportar_metricas(results_wf, "Walk-Forward")
metricas_rw_df = exportar_metricas(results_rw, "Rolling Window")
metricas_ew_df = exportar_metricas(results_ew, "Expanding Window")

print("\n¡Exportación completada!")



Exportando métricas...
Métricas de producto1 (Walk-Forward) exportadas a: metricas/modelo-WMA-metricas_Walk-Forward_producto1.csv
Métricas de producto2 (Walk-Forward) exportadas a: metricas/modelo-WMA-metricas_Walk-Forward_producto2.csv
Métricas de producto1 (Rolling Window) exportadas a: metricas/modelo-WMA-metricas_Rolling_Window_producto1.csv
Métricas de producto2 (Rolling Window) exportadas a: metricas/modelo-WMA-metricas_Rolling_Window_producto2.csv
Métricas de producto1 (Expanding Window) exportadas a: metricas/modelo-WMA-metricas_Expanding_Window_producto1.csv
Métricas de producto2 (Expanding Window) exportadas a: metricas/modelo-WMA-metricas_Expanding_Window_producto2.csv

¡Exportación completada!
